In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymir import mpl_stylesheet
from pymir import mpl_utils
mpl_stylesheet.banskt_presentation(splinecolor = 'black')

In [3]:
from mrashpen.models.normal_means_ash_scaled import NormalMeansASHScaled
from mrashpen.models.normal_means_ash        import NormalMeansASH

def shrinkage_operator(nm, dj, is_prior_scaled = True):
    '''
    posterior expectation of b under NM model
    calculated using Tweedie's formula

    Returns shrinkage operator M(b)
    Dimensions:
        M: vector of size P
        M_bgrad: vector of size P
        M_wgrad: matrix of size P x K
        M_sgrad: vector of size P
    '''
    M       = nm.y + nm.yvar * nm.logML_deriv
    M_bgrad = 1       + nm.yvar * nm.logML_deriv2
    M_wgrad = nm.yvar.reshape(-1, 1) * nm.logML_deriv_wderiv
    if is_prior_scaled:
        M_s2grad = (nm.logML_deriv / dj) + (nm.yvar * nm.logML_deriv_s2deriv)
    else:
        M_s2grad = (nm.logML_deriv + nm.yvar * nm.logML_deriv_s2deriv) / dj
    return M, M_bgrad, M_wgrad, M_s2grad

def invert_Mop(X, b, s2, wk, sk, is_prior_scaled = True, max_iter = 100, tol = 1e-8):
    n, p = X.shape
    dj = np.sum(np.square(X), axis = 0)
    theta = np.zeros(p)
    for itr in range(max_iter):
        if is_prior_scaled:
            nmash = NormalMeansASHScaled(theta, np.sqrt(s2), wk, sk, d = dj)
        else:
            vj = np.sqrt(s2 / dj)
            nmash = NormalMeansASH(theta, vj, wk, sk)
        M_theta, M_theta_bgrad, _, _ = shrinkage_operator(nmash, dj)
        theta_new = theta - (M_theta - b) / M_theta_bgrad
        diff = np.sum(np.square(theta_new - theta))
        print (diff)
        theta = theta_new
        if diff <= tol:
            break
    return theta